# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [14]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,stanley,54.8680,-1.6985,49.71,91,40,4.61,GB,1710262561
1,1,bardai,11.6500,7.2333,98.73,8,92,9.73,NG,1710262562
2,2,puerto natales,-51.7236,-72.4875,59.45,36,40,23.02,CL,1710262562
3,3,kodiak,57.7900,-152.4072,23.50,65,94,9.10,US,1710262562
4,4,waitangi,-43.9535,-176.5597,55.81,76,100,1.99,NZ,1710262562


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display
# Generate a random color for each city
city_colors = {}
for city in city_data_df['City'].unique():
    city_colors[city] = "#{:06x}".format(random.randint(0, 0xFFFFFF))

# Function to get the color for each city
def get_city_color(city):
    return city_colors.get(city, 'gray') 

# Add a 'Color' column to the DataFrame based on the city
city_data_df['Color'] = city_data_df['City'].apply(get_city_color)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng", 
    "Lat",
    geo=True,
    tiles="OSM",
    color='Color',  
    size='Humidity',  
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Color,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df2 = city_data_df[(city_data_df["Max Temp"] > 20) & 
                            (city_data_df["Max Temp"] < 30) & 
                            (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df_drop = city_data_df2.dropna()

# Display sample data
city_data_df_drop

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Color
381,381,fort mcmurray,56.7268,-111.3810,22.59,79,0,9.22,CA,1710262600,#a430dd
459,459,the pas,53.8251,-101.2541,24.82,80,0,8.05,CA,1710262609,#9528ef
557,557,plastun,44.7561,136.2939,21.25,62,0,7.11,RU,1710262619,#bcc133


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_drop[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
381,fort mcmurray,CA,56.7268,-111.3810,79,
459,the pas,CA,53.8251,-101.2541,80,
557,plastun,RU,44.7561,136.2939,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories" :"accommodation.hotel",
"apiKey": geoapify_key,
         "limit" : 20}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
   
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?PARAMS"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fort mcmurray - nearest hotel: Nomad Hotel and Suites
the pas - nearest hotel: Wescana Inn
plastun - nearest hotel: Россияночка


,City,Country,Lat,Lng,Humidity,Hotel Name
381,fort mcmurray,CA,56.7268,-111.3810,79,Nomad Hotel and Suites
459,the pas,CA,53.8251,-101.2541,80,Wescana Inn
557,plastun,RU,44.7561,136.2939,62,Россияночка


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Generate a random color for each city
city_colors = {}
for city in hotel_df['City'].unique():
    city_colors[city] = "#{:06x}".format(random.randint(0, 0xFFFFFF))

# Function to get the color for each city
def get_city_color(city):
    return city_colors.get(city, 'gray') 

# Add a 'Color' column to the DataFrame based on the city
hotel_df['Color'] = hotel_df['City'].apply(get_city_color)

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols =["Hotel Name","Country"],
    geo = True,
    tiles = "OSM",
    color='Color'
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Color,Hotel Name,Country)